In [1]:
#Preflop analyzer


In [2]:
#Goal create a Faise First In(RFI) table with position 
 #howto - need to parse my hands, then determine what position im in, and determine if im the RFI
 #howto - then, i need to matplotlib 6 or so plots from the individual positions
         #- i would like this to be in the standard (A-2 * A-2) plot coloring the played range 

#Goal 2 - take the data and report some sort of percentage of play for hand per position, otherwise we may misrepresent a super large range of hands 
 #howto - we need a boolean average of hand per position reported in our figure 
#Goal 

In [3]:
import pandas as pd

In [7]:
#ignitionHandHistoryFile = input("Text file dir:")
handhistoryfilename = input("file dir:")
handhistoryfilename.replace("\\","\\\\")
#handhistoryfilename.replace('','')                            
ignitionHands = open(handhistoryfilename,"r").read()




file dir:C:\programming\preflop2\IPHH\3.txt


In [141]:
testHand = ignitionHands.split("Ignition Hand")[0]
print(testHand)

In [107]:
#completed functions 

def parseHandPos(preflopLine):
    #accept 1 line of preflop text 
    #returns hand and Position info
    
    if preflopLine.find("Card dealt to a spot") == -1: #this would mean this isnt a card assignemnt line #mistakes 
        positionInfo,leftcard,rightcard = None,None,None
    else: #text parse out 
        
        preflopLinewMeRemoved = preflopLine.replace(' [ME] ','')
        positionInfo = preflopLinewMeRemoved[:preflopLinewMeRemoved.find(":")-1]
        leftcard = preflopLinewMeRemoved[preflopLinewMeRemoved.find("[")+1:preflopLinewMeRemoved.find("[")+3]
        rightcard = preflopLinewMeRemoved[preflopLinewMeRemoved.find("[")+4:preflopLinewMeRemoved.find("[")+6]
    
    return positionInfo,leftcard,rightcard #cards have no fucntional order

def findSubstringInList(searchList, substring):
    #this useful fuction will return the index of the first list index that contains the substring
    
    if any(substring in string for string in searchList):
        return [idx for idx, s in enumerate(searchList) if substring in s][0] #returns the index location
    else:
        return -1 # returns -1 if the substring is not in the list

def preflopChunkParse(handHistory):
    ##assume that the last preflop aciton is the line before the "*** FLOP ***"
    handHistoryLines = handHistory.splitlines()
    
    preflopEndPhrase = "*** FLOP ***"
    preflopStartPhrase = "Card dealt to a spot"
    #using the phrase that starts the next phase
    preflopEnd =  findSubstringInList(handHistoryLines,preflopEndPhrase)
    
    #find the first list index in the reverse list that contains the phrase for card delt
    #this should be the last thing done before the preflop aciton 
    preflopStart = len(handHistoryLines) - findSubstringInList(handHistoryLines[::-1],preflopStartPhrase)
    
    #find the beginning and ending line
    preflopHandHistoryLines = handHistoryLines[preflopStart:preflopEnd]
    return preflopHandHistoryLines

def RFIAction(preflopHandHistoryLines):
    #expecting input of lines from the preflop
    
    RFIEndPhrase = "Raises"
    
    endingIndex = findSubstringInList(preflopHandHistoryLines, RFIEndPhrase)
    
    RFIActionLines = preflopHandHistoryLines[endingIndex]
    
    return RFIActionLines

def getAllLinesContainingSubString(listOfStrings, substring):
    
    return [s for s in listOfStrings if substring in s]



def spliceDeal(handHistory):
    handHistoryLines = handHistory.splitlines() 
    cardDealPhrase = "Card dealt to a spot"
    return getAllLinesContainingSubString(handHistoryLines, cardDealPhrase)


def createHoleCardDict(handHistory):
    
    #first reduce the Hand History to a list of relevant dealing card to position list
    dealtCardLines = spliceDeal(handHistory)
    
    #this line creats a dictionary by taking each line of the dealing action 
    #then it runs throught the parseHandpositions which give it the pos dict to the cards
    holeCardByPos = {parseHandPos(dealCardLine)[0]:(parseHandPos(dealCardLine)[1],parseHandPos(dealCardLine)[2]) for dealCardLine in dealtCardLines}
        
    return holeCardByPos

def getRFIPositionHand(agrigateHandHistory):
    #feed in Hand history output 
    #we want a dataframe that has Position and hand for every hand fed through a string
    
    #get line with RFI action 
    RFIAcitonLine = RFIAction(agrigateHandHistory.splitlines())
    
    preflopLinewMeRemoved = RFIAcitonLine.replace(' [ME] ','')
    RFIPosition = preflopLinewMeRemoved[:preflopLinewMeRemoved.find(":")-1]
    
    #use the line to get position and then feed into dictionary
    
    holeCardByPos = createHoleCardDict(agrigateHandHistory)
    
    hand = (holeCardByPos[RFIPosition])
    
    
    return RFIPosition, hand 
    


In [116]:
###-
###This takes the session Hand history and puts it into one file for processing
###---------------------------------------------------------------------------

from os import listdir
from os.path import isfile, join

def mergeTextToWriteDestination(filenames,Dir):
    fileOutputPath = input("path to output files")
    with open(fileOutputPath, 'w') as outfile:
        for fname in filenames:
            with open(Dir + fname) as infile:
                outfile.write(infile.read())
    pass

def getAllFilesFromPath(directoryPath):
    return [f for f in listdir(directoryPath) if isfile(join(directoryPath, f))]

#Main Rrun
#this could be created as a function 
def sessionFilestoOneFile():
    mypath = input("Path to Files")
    AllFilePaths = getAllFilesFromPath(mypath)

    mergeTextToWriteDestination(AllFilePaths, mypath)
    pass







In [118]:
def basicFileReader(): #rename this is aweful
    handhistoryfilename = input("file dir:")
    handhistoryfilename.replace("\\","\\\\")
    #handhistoryfilename.replace('','')                            
    handHistoryText = open(handhistoryfilename,"r").read()
    return handHistoryText



In [149]:

#position_Headers = ['Dealer' , 'Small Blind' , 'Big Blind', 'UTG', 'UTG+1', 'UTG+2', 'UTG+3', 'UTG+4', 'UTG+5']

Header1 = 'Positions'
Header2 = 'Hand'

position_Headers = [Header1, Header2]
RFITable = pd.DataFrame(columns = position_Headers)


bulkHandHistory = basicFileReader()
#iterate over Hands
bulkHandHistoryLines = bulkHandHistory.split("Ignition Hand")

hand = bulkHandHistoryLines[0]
print(hand)
count=0
for hand in bulkHandHistoryLines:
    print(count)
    count+=1
    if len(hand)>10:
        d = {Header1: getRFIPositionHand(hand)[0], Header2 : (getRFIPositionHand(hand)[1],)}
        newRFIData = pd.DataFrame(d)
        RFITable.append(newRFIData)



RFITable.head()


file dir:C:\programming\preflop2\IPHH\Data.txt

0
1


KeyError: ''

In [136]:
bulkHandHistory = basicFileReader()
print(bulkHandHistory[0:201])

file dir:C:\programming\preflop2\IPHH\Data.txt
Ignition Hand #39152


In [147]:
print((bulkHandHistory.split("Ignition Hand")[1]))

Header1 = 'Positions'
Header2 = 'Hand'

d = {Header1: getRFIPositionHand(hand)[0], Header2 : (getRFIPositionHand(hand)[1],)}

 #3915239935 TBL#21035758 HOLDEM No Limit - 2020-04-04 15:54:57
Seat 1: UTG+2 ($3.43 in chips)
Seat 2: UTG+3 ($46.94 in chips)
Seat 3: Dealer ($10 in chips)
Seat 4: Small Blind ($1.84 in chips)
Seat 5: Big Blind [ME] ($10 in chips)
Seat 8: UTG ($3.19 in chips)
Seat 9: UTG+1 ($11.54 in chips)
Dealer : Set dealer [3] 
Small Blind : Small Blind $0.05 
Big Blind  [ME] : Big blind $0.10 
*** HOLE CARDS ***
UTG+2 : Card dealt to a spot [As 7h] 
UTG+3 : Card dealt to a spot [3s 6c] 
Dealer : Card dealt to a spot [3h Qs] 
Small Blind : Card dealt to a spot [Ts 8h] 
Big Blind  [ME] : Card dealt to a spot [9c 2s] 
UTG : Card dealt to a spot [2h Qh] 
UTG+1 : Card dealt to a spot [6h Qd] 
UTG : Folds
UTG+1 : Folds
UTG+2 : Calls $0.10 
UTG+3 : Folds
Dealer : Folds
Small Blind : Calls $0.05 
Big Blind  [ME] : Checks
*** FLOP *** [6d Ah 7s]
Small Blind : Checks
Seat stand
Table leave user
Big Blind  [ME] : Checks
UTG+2 : Bets $0.15 
Table enter user
Seat sit down
Table deposit $10 
Small Blind : Fold